In [ ]:
import json
from typing import Dict, List

class Book:
    def __init__(self, title: str, author: str, isbn: str):
        self._title = title
        self._author = author
        self._isbn = isbn
        self._is_borrowed = False

    def borrow_book(self) -> bool:
        if not self._is_borrowed:
            self._is_borrowed = True
            return True
        else:
            return False

    def return_book(self) -> bool:
        if self._is_borrowed:
            self._is_borrowed = False
            return True
        else:
            return False

    def get_status(self):
        self._status = "Borrowed" if self._is_borrowed else "Available"
        return self._status

    def __str__(self) -> str:
        return (f"Title:{self._title}\n"
                f"Author:{self._author}\n"
                f"ISBN:{self._isbn}\n"
                f"Status:{self.get_status()}")

    def to_dict(self) -> dict:
        return {
            "Title": self._title,
            "Author": self._author,
            "ISBN": self._isbn,
            "Status": self.get_status()
        }

class User:
    def __init__(self, name: str, user_id: str):
        self._name = name
        self._user_id = user_id
        self._borrowed_books_isbns = []

    def add_borrowed_book_isbn(self, isbn: str) -> None:
        if isbn not in self._borrowed_books_isbns:
            self._borrowed_books_isbns.append(isbn)
            print(f"ISBN {isbn} added.")
        else:
            print(f"ISBN {isbn} is already borrowed.")

    def remove_borrowed_book_isbn(self, isbn: str) -> None:
        if isbn in self._borrowed_books_isbns:
            self._borrowed_books_isbns.remove(isbn)
            print(f"ISBN {isbn} removed.")
        else:
            print(f"ISBN {isbn} is not borrowed.")

    def __str__(self) -> str:
        return (f"User:{self._name}\n"
                f"User Id:{self._user_id}\n"
                f"Borrowed books:{len(self._borrowed_books_isbns)}\n")

    def to_dict(self) -> dict:
        return {
            "User": self._name,
            "User Id": self._user_id,
            "Borrowed books": self._borrowed_books_isbns
        }

class Library:
    def __init__(self, book_file: str = 'books.json', user_file: str = 'users.json'):
        self._book_file = book_file
        self._user_file = user_file
        self._users: Dict[str, User] = {}
        self._books: Dict[str, Book] = {}
        self._load_data()
        self._load_user()

    def _load_data(self) -> None:
        try:
            with open(self._book_file, 'r') as bf:
                book_data = json.load(bf)
                for book in book_data:
                    b = Book(book['Title'], book['Author'], book['ISBN'])
                    if book.get("Status") == 'Borrowed':
                        b.borrow_book()
                    self._books[book['ISBN']] = b
        except FileNotFoundError:
            print(f"{self._book_file} not found.")

    def _load_user(self) -> None:
        try:
            with open(self._user_file, 'r') as uf:
                user_data = json.load(uf)
                for user in user_data:
                    u = User(user['User'], user['User Id'])
                    for isbn in user.get('Borrowed books', []):
                        u.add_borrowed_book_isbn(isbn)
                    self._users[user['User Id']] = u
        except FileNotFoundError:
            print(f"{self._user_file} not found.")

    def _save_data(self) -> None:
        try:
            with open(self._book_file, 'w') as bf:
                books_list = [book.to_dict() for book in self._books.values()]
                json.dump(books_list, bf, indent=4)
        except Exception as e:
            print(f"Error saving book data: {e}")

        try:
            with open(self._user_file, 'w') as uf:
                users_list = [user.to_dict() for user in self._users.values()]
                json.dump(users_list, uf, indent=4)
        except Exception as e:
            print(f"Error saving user data: {e}")

    def add_book(self, book: Book) -> bool:
        if book._isbn in self._books:
            print(f"Book with ISBN {book._isbn} already exists.")
            return False
        else:
            self._books[book._isbn] = book
            self._save_data()
            print(f"Book '{book._title}' added successfully.")
            return True

    def remove_book(self, isbn: str) -> bool:
        if isbn not in self._books:
            print(f"Book with ISBN {isbn} doesn't exist.")
            return False

        book = self._books[isbn]

        if book.get_status() == "Borrowed":
            print(f"Cannot remove the book {isbn}, it is currently borrowed.")
            return False
        else:
            del self._books[isbn]
            self._save_data()
            print(f"Book with ISBN {isbn} removed successfully.")
            return True

    def register_user(self, user: User) -> bool:
        if user._user_id in self._users:
            print(f"User ID {user._user_id} already registered")
            return False
        else:
            self._users[user._user_id] = user
            self._save_data()
            print(f"User ID {user._user_id} registered successfully.")
            return True

    def remove_user(self, user_id: str) -> bool:
        if user_id not in self._users:
            print(f"User ID {user_id} doesn't exist.")
            return False

        user = self._users[user_id]

        if user._borrowed_books_isbns:
            print(f"Cannot remove User ID {user_id}, they have currently borrowed books.")
            return False
        else:
            del self._users[user_id]
            self._save_data()
            print(f"User ID {user_id} removed successfully.")
            return True

    def borrow_book(self, isbn: str, user_id: str) -> bool:
        if isbn not in self._books:
            print(f"Book with ISBN {isbn} not found.")
            return False

        if user_id not in self._users:
            print(f"User ID {user_id} not found.")
            return False

        book = self._books[isbn]
        user = self._users[user_id]

        if not book.borrow_book():
            print(f"Book with ISBN {isbn} is already borrowed.")
            return False

        user.add_borrowed_book_isbn(isbn)
        self._save_data()
        print(f"Book with ISBN {isbn} successfully borrowed by User ID {user_id}.")
        return True

    def return_book(self, isbn: str, user_id: str) -> bool:
        if isbn not in self._books:
            print(f"Book with ISBN {isbn} not found.")
            return False

        if user_id not in self._users:
            print(f"User ID {user_id} not found.")
            return False

        user = self._users[user_id]
        book = self._books[isbn]

        if isbn not in user._borrowed_books_isbns:
            print(f"User ID {user_id} did not borrow book with ISBN {isbn}.")
            return False

        if not book.return_book():
            print(f"Book with ISBN {isbn} could not be returned.")
            return False

        user.remove_borrowed_book_isbn(isbn)
        self._save_data()
        print(f"Book with ISBN {isbn} successfully returned by User ID {user_id}.")
        return True

    def search_book(self, query: str) -> list[Book]:
        query_lower = query.lower()
        results = []

        for book in self._books.values():
            if (query_lower in book._title.lower() or
                query_lower in book._author.lower() or
                query in book._isbn):
                results.append(book)

        return results

    def display_all_books(self, show_available_only: bool = False) -> None:
        for book in self._books.values():
            if show_available_only and book.get_status() != "Available":
                continue
            print(book)
            print("-" * 30)

    def display_all_users(self) -> None:
        for user in self._users.values():
            print(user)
            print("-" * 30)

    def display_user_borrowed_books(self, user_id: str) -> None:
        user = self._users.get(user_id)
        if not user:
            print(f"User ID {user_id} not found.")
            return

        if not user._borrowed_books_isbns:
            print(f"User ID {user_id} has not borrowed any books.")
            return

        print(f"Books borrowed by User ID {user_id}:")
        for isbn in user._borrowed_books_isbns:
            book = self._books.get(isbn)
            if book:
                print(book)
                print("-" * 30)

    def get_all_borrowed_books(self) -> None:
        borrowed_books = [book for book in self._books.values() if book.get_status() == "Borrowed"]

        if not borrowed_books:
            print("No books are currently borrowed.")
            return

        print("All Currently Borrowed Books:")
        for book in borrowed_books:
            print(book)
            print("-" * 30)

    def run(self):
        while True:
            print("\nLibrary Menu:")
            print("1. Add Book")
            print("2. Register User")
            print("3. Borrow Book")
            print("4. Return Book")
            print("5. Display All Books")
            print("6. Search Book")
            print("7. Display All Users")
            print("8. Display User's Borrowed Books")
            print("9. Display All Borrowed Books")
            print("10. Exit")

            choice = input("Enter your choice (1-10): ")

            if choice == '1':
                title = input("Enter book title: ")
                author = input("Enter book author: ")
                isbn = input("Enter book ISBN: ")
                book = Book(title, author, isbn)
                self.add_book(book)

            elif choice == '2':
                name = input("Enter user name: ")
                user_id = input("Enter user ID: ")
                user = User(name, user_id)
                self.register_user(user)

            elif choice == '3':
                isbn = input("Enter ISBN of the book to borrow: ")
                user_id = input("Enter your user ID: ")
                self.borrow_book(isbn, user_id)

            elif choice == '4':
                isbn = input("Enter ISBN of the book to return: ")
                user_id = input("Enter your user ID: ")
                self.return_book(isbn, user_id)

            elif choice == '5':
                only_available = input("Show only available books? (y/n): ").strip().lower() == 'y'
                self.display_all_books(show_available_only=only_available)

            elif choice == '6':
                query = input("Enter title/author/ISBN to search: ")
                results = self.search_book(query)
                if results:
                    print("Search Results:")
                    for book in results:
                        print(book)
                        print("-" * 30)
                else:
                    print("No matching books found.")

            elif choice == '7':
                self.display_all_users()

            elif choice == '8':
                user_id = input("Enter user ID: ")
                self.display_user_borrowed_books(user_id)

            elif choice == '9':
                self.get_all_borrowed_books()

            elif choice == '10':
                print("Exiting the Library System. Goodbye!")
                break

            else:
                print("Invalid choice. Please enter a number from 1 to 10.")

def main():
    library = Library()
    library.run()

main()


ISBN 9780199291151 added.
ISBN 9780393338102 added.

Library Menu:
1. Add Book
2. Register User
3. Borrow Book
4. Return Book
5. Display All Books
6. Search Book
7. Display All Users
8. Display User's Borrowed Books
9. Display All Borrowed Books
10. Exit


Enter your choice (1-10):  1
Enter book title:  The wings of fire
Enter book author:  Dr. APJ Abdul Kalam
Enter book ISBN:  9780393392131


Book 'The wings of fire' added successfully.

Library Menu:
1. Add Book
2. Register User
3. Borrow Book
4. Return Book
5. Display All Books
6. Search Book
7. Display All Users
8. Display User's Borrowed Books
9. Display All Borrowed Books
10. Exit


Enter your choice (1-10):  2
Enter user name:  Ayushi
Enter user ID:  108


User ID 108 registered successfully.

Library Menu:
1. Add Book
2. Register User
3. Borrow Book
4. Return Book
5. Display All Books
6. Search Book
7. Display All Users
8. Display User's Borrowed Books
9. Display All Borrowed Books
10. Exit


...

User:Dr. Ananya Sharma
User Id:U100
Borrowed books:1

------------------------------
User:Prof. Rajeev Mehta
User Id:U101
Borrowed books:1

------------------------------
User:Sneha Verma
User Id:U102
Borrowed books:0

------------------------------
User:Rishabh Singh Yadav
User Id:U105
Borrowed books:0

------------------------------
User:Ayushi
User Id:108
Borrowed books:0

------------------------------

Library Menu:
1. Add Book
2. Register User
3. Borrow Book
4. Return Book
5. Display All Books
6. Search Book
7. Display All Users
8. Display User's Borrowed Books
9. Display All Borrowed Books
10. Exit


Enter your choice (1-10):  8
Enter user ID:  U100


Books borrowed by User ID U100:
Title:The Selfish Gene
Author:Richard Dawkins
ISBN:9780199291151
Status:Borrowed
------------------------------

Library Menu:
1. Add Book
2. Register User
3. Borrow Book
4. Return Book
5. Display All Books
6. Search Book
7. Display All Users
8. Display User's Borrowed Books
9. Display All Borrowed Books
10. Exit


Enter your choice (1-10):  9


All Currently Borrowed Books:
Title:The Selfish Gene
Author:Richard Dawkins
ISBN:9780199291151
Status:Borrowed
------------------------------
Title:The Elegant Universe
Author:Brian Greene
ISBN:9780393338102
Status:Borrowed
------------------------------

Library Menu:
1. Add Book
2. Register User
3. Borrow Book
4. Return Book
5. Display All Books
6. Search Book
7. Display All Users
8. Display User's Borrowed Books
9. Display All Borrowed Books
10. Exit
